In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3


In [3]:
path = '/Users/SebastianBerko/Documents/Bank Term Deposit Prediction/data.db'
conn = sqlite3.connect(path)
cursor = conn.cursor()

In [ ]:
# Find the table names and exclude system file
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = [row[0] for row in cursor.fetchall() if row[0] != 'sqlite_sequence']  # Exclude system tables if needed

# Dictionary to store DataFrames
dataframes = {}

# Loop through each table and fetch data
for table in tables:
    query = f"SELECT * FROM {table}"
    cursor.execute(query)
    rows = cursor.fetchall()
    
    # Get column names for the DataFrame
    column_names = [desc[0] for desc in cursor.description]
    
    # Create DataFrame and store it in the dictionary
    dataframes[table] = pd.DataFrame(rows, columns=column_names)

# Access individual DataFrames
accounts = dataframes['Accounts']
campaigns = dataframes['Campaigns']
clients = dataframes['Clients']
outcomes = dataframes['Outcomes']




In [5]:
# Rename columns for clarity
renamed_dataframes = {
    "accounts": accounts.rename(columns={"id": "account_id"}),
    "campaigns": campaigns.rename(columns={"id": "campaign_id"}),
    "clients": clients.rename(columns={"id": "client_id"}),
    "outcomes": outcomes.rename(columns={"id": "outcome_id"}),
}

# Perform successive merges
merged_data = (
    renamed_dataframes["clients"]
    .merge(renamed_dataframes["accounts"], on="client_id", suffixes=("", "_drop"))
    .merge(renamed_dataframes["campaigns"], on="account_id", suffixes=("", "_drop"))
    .merge(renamed_dataframes["outcomes"], on="campaign_id", suffixes=("", "_drop"))
)

# Drop unwanted duplicate columns created by merges
columns_to_drop = [col for col in merged_data.columns if col.endswith("_drop")]
bank_data = merged_data.drop(columns=columns_to_drop)

# Rename any necessary columns
bank_data = bank_data.rename(columns={"campaign_id": "campaign_id_x"})

# Display or further process `bank_data`
bank_data.head()


,client_id,age,job,marital,education,account_id,in_default,balance,housing,loan,...,campaign_id_x,day,month,duration,campaign,pdays,previous,outcome_id,poutcome,y
0,1000583,43,management,married,tertiary,19298,no,-127.0,no,no,...,19298,6,aug,400,2,-1,0,19298,unknown,unknown
1,1000708,45,management,divorced,secondary,18266,no,51.0,yes,no,...,18266,31,jul,67,8,-1,0,18266,unknown,unknown
2,1000951,35,management,single,tertiary,24000,no,385.0,yes,no,...,24000,29,aug,168,2,-1,0,24000,unknown,unknown
3,1001250,29,blue-collar,single,secondary,5448,no,1012.0,yes,no,...,5448,23,may,926,1,-1,0,5448,unknown,unknown
4,1001280,50,blue-collar,married,unknown,19620,no,123.0,yes,no,...,19620,7,aug,276,2,-1,0,19620,unknown,unknown
